In [1]:
!pip install langchain
!pip install openai
!pip install PyPDF2
!pip install faiss-cpu
!pip install tiktoken

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [6]:
from PyPDF2 import PdfReader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import ElasticVectorSearch, Pinecone, Weaviate, FAISS

from langchain_openai import AzureChatOpenAI
from langchain_openai import AzureOpenAIEmbeddings

In [ ]:
# Get your API keys from openai, you will need to create an account.
# Here is the link to get the keys: https://platform.openai.com/account/billing/overview
import os
os.environ["OPENAI_API_KEY"] = "YOUR-OPENAI-API-KEY"

In [7]:
import os

#Setting Environment variable
os.environ["AZURE_OPENAI_API_VERSION"] = "2023-07-01-preview"
os.environ["AZURE_OPENAI_ENDPOINT"] = 'https://dskumar.openai.azure.com/'
os.environ["AZURE_OPENAI_API_KEY"] ="62855d6dd08945819bf83aee0c104127"
os.environ["AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"] ="DskumarDeployment"
os.environ['OPENAI_TYPE']="Azure"
os.environ["LLM_MODEL"] = "gpt-35-turbo-16k"
os.environ["LLM_EMBEDDING_MODEL"] = "dskumar-text-embedding-ada-002"

In [8]:
from langchain_core.messages import AIMessage,SystemMessage,HumanMessage
from langchain_openai import AzureChatOpenAI

model = AzureChatOpenAI(
    openai_api_version=os.environ["AZURE_OPENAI_API_VERSION"],
    azure_deployment=os.environ["AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"],
)

In [9]:
# location of the pdf file/files.
reader = PdfReader(r'C:\Users\JANU\OneDrive\Documents\GitHub\ML-AI\Codes\Projects&POC\Citizen_POC\Citizens bank-2023.pdf')

In [6]:
reader

In [10]:
# read data from the file and put them into a variable called raw_text
raw_text = ''
for i, page in enumerate(reader.pages):
    text = page.extract_text()
    if text:
        raw_text += text

In [ ]:
# raw_text

In [11]:
raw_text[:99]

'UNITED STATES\nSECURITIES AND EXCHANGE COMMISSION\nWashington, D.C. 20549\nFORM 10-K\n☒ ANNUAL REPORT P'

In [12]:
# We need to split the text that we read into smaller chunks so that during information retreival we don't hit the token size limits.

text_splitter = CharacterTextSplitter(
    separator = "\n",
    chunk_size = 1000,
    chunk_overlap  = 200,
    length_function = len,
)
texts = text_splitter.split_text(raw_text)

In [10]:
len(texts)

933

In [11]:
texts[0]

'UNITED STATES\nSECURITIES AND EXCHANGE COMMISSION\nWashington, D.C. 20549\nFORM 10-K\n☒ ANNUAL REPORT PURSUANT TO SECTION 13 OR 15(d)\nOF THE SECURITIES EXCHANGE ACT OF 1934\nFor the Fiscal Year Ended\nDecember 31, 2023\n☐ TRANSITION REPORT PURSUANT TO SECTION 13 OR 15(d)\nOF THE SECURITIES EXCHANGE ACT OF 1934\nFor the T ransition Period From\n(Not Applicable)\nCommission File Number 001-36636\n(Exact name of the registrant as specified in its charter)\nDelaware 05-0412693\n(State or Other Jurisdiction of\nIncorporation or Organization)(I.R.S. Employer\nIdentification Number)\nOne Citizens Plaza, Providence, RI 02903\n(Address of principal executive offices, including zip code)\n(203) 900-6715\n(Registrant’s telephone number, including area code)\nSecurities registered pursuant to Section 12(b) of the Act:\nTitle of each class Trading symbol(s) Name of each exchange on which registered\nCommon stock, $0.01 par value per share CFG New York Stock Exchange'

In [12]:
texts[2]

'Indicate by check mark if the registrant is not required to file reports pursuant to Section 13 or Section 15(d) of the Act. ☐ Yes ☑ No\nIndicate by check mark whether the Registrant (1) has filed all reports required to be filed by Section 13 or 15(d) of the Securities Exchange Act of 1934 during the preceding 12 months (or for such\nshorter period that the registrant was required to file such reports) and (2) has been subject to such filing requirements for the past 90 days. ☑ Yes ☐ No\nIndicate by check mark whether the registrant has submitted electronically every Interactive Data File required to be submitted pursuant to Rule 405 of Regulation S-T (§232.405 of this chapter) during\nthe preceding 12 months (or for such shorter period that the registrant was required to submit such files). ☑ Yes ☐ No'

In [13]:
# Download embeddings from OpenAI
#embeddings = OpenAIEmbeddings()


from langchain_openai import AzureChatOpenAI
from langchain_openai import AzureOpenAIEmbeddings

#embeddings
embeddings = AzureOpenAIEmbeddings(model = os.environ["LLM_EMBEDDING_MODEL"],)


In [14]:
#vectorstore
vectorstore = FAISS.from_texts(texts, embeddings)

In [17]:
vectorstore

In [15]:
#retriver
retriever = vectorstore.as_retriever()

In [19]:
from langchain.chains.question_answering import load_qa_chain
from langchain.llms import AzureOpenAI

In [20]:
chain = load_qa_chain(model, chain_type="stuff")

In [21]:
query = "What are the bank’s core products and services?"
docs = vectorstore.similarity_search(query)
chain.run(input_documents=docs, question=query)

C:\Users\JANU\AppData\Roaming\Python\Python311\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


"The bank's core products and services include:\n\n1. Consumer Banking: This segment serves consumer customers and small businesses with a range of products and services such as deposits, mortgage and home equity lending, credit cards, small business loans, wealth management, and investment services. They also offer education and point-of-sale finance loans. Additionally, they have a multi-channel distribution network with branches, ATMs, and online/mobile platforms.\n\n2. Commercial Banking: This segment primarily serves companies and institutions with annual revenues ranging from $25 million to over $3.0 billion. They offer a broad range of financial products and solutions, including lending and leasing, deposit and treasury management services, foreign exchange, interest rate and commodity risk management solutions, loan syndication, corporate finance, merger and acquisition services, and debt and equity capital markets capabilities.\n\nIt's important to note that specific details a

In [22]:
query = "What is the company's return on equity (ROE)?"
docs = vectorstore.similarity_search(query)
chain.run(input_documents=docs, question=query)

"The company's return on equity (ROE) is not explicitly stated in the provided context."

In [23]:
query = "what is the net income of the company for 2023?"
docs = vectorstore.similarity_search(query)
chain.run(input_documents=docs, question=query)

'The net income of the company for 2023 is $1,608 million.'

In [24]:
query = "what is the shareholder’s equity of the company for 2023?"
docs = vectorstore.similarity_search(query)
chain.run(input_documents=docs, question=query)

"The shareholder's equity of the company for 2023 is $24,342."

In [25]:
query = "what is the stock symbol name of the company?"
docs = vectorstore.similarity_search(query)
chain.run(input_documents=docs, question=query)

'The stock symbol name of the company is CFG, and it is listed on the New York Stock Exchange.'

In [26]:
query = "ROE will be calculated based on net income of the company/shareholder’s equity of the company. What is the ROE of the company for the year 2023? "
docs = vectorstore.similarity_search(query)
chain.run(input_documents=docs, question=query)

'The ROE (Return on Equity) for the year 2023 is not explicitly stated in the provided information.'

In [27]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(
    memory_key="chat_history",
    chat_memory=chat_history,  # this is your persistence strategy subclass of `BaseChatMessageHistory`
    output_key="answer",
    return_messages=True
)

qa = ConversationalRetrievalChain.from_llm(
    llm=model,
    memory=memory,
    retriever=retriever, 
    return_source_documents=True,
    return_generated_question=True,
    verbose=True
)

NameError: name 'chat_history' is not defined

In [30]:
from langchain.tools.retriever import create_retriever_tool

In [31]:
retriever_tool = create_retriever_tool(
    retriever,
    "input",
    "use this document for all the question and answers",
)

In [32]:
tools = [retriever_tool]

In [33]:
from langchain_openai import AzureChatOpenAI

chat_model = AzureChatOpenAI(
    openai_api_version=os.environ["AZURE_OPENAI_API_VERSION"],
    azure_deployment=os.environ["AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"],
)


In [74]:
!pip install langchainhub

Defaulting to user installation because normal site-packages is not writeable


In [16]:
from langchain import hub

# Get the prompt to use - you can modify this! - https://smith.langchain.com/hub/hwchase17/openai-functions-agent
prompt = hub.pull("rlm/rag-prompt")
#hub.pull("hwchase17/openai-functions-agent")
prompt.messages

[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"))]

In [35]:
from langchain.agents import create_tool_calling_agent

agent = create_tool_calling_agent(chat_model, tools, prompt)

In [36]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [37]:
agent_executor.invoke({"input": "hi!"})



> Entering new AgentExecutor chain...
Hello! How can I assist you today?

> Finished chain.


{'input': 'hi!', 'output': 'Hello! How can I assist you today?'}

In [38]:
agent_executor.invoke({"input": "What are the bank’s core products and services?"})



> Entering new AgentExecutor chain...

Invoking: `input` with `{'query': 'bank core products and services'}`


We are a BHC incorporated under Delaware state law in 1984 and our primary federal regulator is the FRB. CBNA is our banking
subsidiary, whose primary federal regulator is the OCC.
Business Segments
We manage our business through two primary business segments: Consumer Banking and Commercial Banking. Our activities outside
these segments are classified as Non-Core or Other. Non-Core includes our indirect auto and certain purchased consumer loan portfolios that
we discontinued the origination of as part of our recently announced balance sheet optimization strategy. Other includes treasury activities,
wholesale funding, the securities portfolio, community development assets, and other unallocated assets, liabilities, capital, revenues, provision
(benefit) for credit losses and expenses, including income tax expense. For additional information regarding our business segments se

{'input': 'What are the bank’s core products and services?',
 'output': "The bank's core products and services are:\n\n1. Consumer Banking: This includes a full range of banking, lending, savings, wealth management, and small business offerings. It provides an integrated experience that includes mobile and online banking, a customer contact center, ATMs, and branches.\n\n2. Commercial Banking: This segment offers lending and leasing, deposit and treasury management services, foreign exchange, risk management solutions, loan syndication, corporate finance, merger and acquisition, and debt and equity capital markets capabilities. It primarily serves companies and institutions with annual revenues of $25 million to more than $3.0 billion.\n\nPlease note that this information is based on the general understanding and may vary depending on the specific bank."}

In [39]:
agent_executor.invoke({"input": "what is the net income of the company for 2023?", "chat_history": []})



> Entering new AgentExecutor chain...

Invoking: `input` with `{'query': 'net income 2023'}`


Year Ended December 31, 2023
(dollars in millions)Consumer
BankingCommercial
Banking Non-Core Other Consolidated
Net interest income $4,187 $2,292 ($129) ($109) $6,241 
Noninterest income 1,067 784 — 132 1,983 
Total revenue 5,254 3,076 (129) 23 8,224 
Noninterest expense 3,542 1,295 123 547 5,507 
Profit (loss) before provision (benefit) for credit losses 1,712 1,781 (252) (524) 2,717 
Provision (benefit) for credit losses 280 250 78 79 687 
Income (loss) before income tax expense (benefit) 1,432 1,531 (330) (603) 2,030 
Income tax expense (benefit) 373 378 (86) (243) 422 
Net income (loss) $1,059 $1,153 ($244) ($360) $1,608 
Total average assets $72,693 $76,028 $13,745 $59,755 $222,221 
Year Ended December 31, 2022
(dollars in millions)Consumer
BankingCommercial
Banking Non-Core Other Consolidated
Net interest income $3,649 $2,103 $378 ($118) $6,012 
Noninterest income 1,063 845 — 101 2,0

{'input': 'what is the net income of the company for 2023?',
 'chat_history': [],
 'output': 'The net income of the company for 2023 is $1,608 million.'}

In [102]:
agent_executor.invoke({"input": "what is the shareholder’s equity of the company for 2023?", "chat_history": []})



> Entering new AgentExecutor chain...

Invoking: `ROE` with `{'query': 'shareholder’s equity of the company for 2023'}`


Derivative liabilities 1,562 1,909 
Long-term borrowed funds 13,467 15,887 
Other liabilities 4,746 4,520 
TOTAL LIABILITIES 197,622 203,043 
Commitments and Contingencies (refer to Note 19)
STOCKHOLDERS’ EQUITY:
Preferred Stock:
$25.00 par value,100,000,000 shares authorized; 2,050,000 shares issued and outstanding at December 31, 2023 and
2022 2,014 2,014 
Common stock:
$0.01 par value, 1,000,000,000 shares authorized; 647,829,720 shares issued and 466,418,055 shares outstanding at
December 31, 2023 and 645,220,018 shares issued and 492,282,158 shares outstanding at December 31, 2022 6 6 
Additional paid-in capital 22,250 22,142 
Retained earnings 9,816 9,159 
Treasury stock, at cost, 181,411,665 and 152,937,860 shares at December 31, 2023 and 2022, respectively (5,986) (5,071)
Accumulated other comprehensive income (loss) (3,758) (4,560)
TOTAL STOCKHOLDERS’ EQU

{'input': 'what is the shareholder’s equity of the company for 2023?',
 'chat_history': [],
 'output': "The shareholder's equity of the company for 2023 is $24,342 million."}

In [40]:
agent_executor.invoke({"input": "what is the stock symbol name of the company","chat_history": []})



> Entering new AgentExecutor chain...
To provide you with the stock symbol name of a company, I need the name of the company you are referring to. Can you please provide me with the name of the company?

> Finished chain.


{'input': 'what is the stock symbol name of the company',
 'chat_history': [],
 'output': 'To provide you with the stock symbol name of a company, I need the name of the company you are referring to. Can you please provide me with the name of the company?'}

In [41]:
agent_executor.invoke({"input": "what is the ROE for 2023?",
                       "chat_history":[HumanMessage(content="ROE can be calculated using the formula net income divided by shareholder's equity"),
            AIMessage(content="Sure, I'll calculate ROE using this formula, ROE = net income of the company/shareholder's equity of the company"),
                                      ]
                      }
                     )



> Entering new AgentExecutor chain...

Invoking: `input` with `{'query': 'ROE for 2023'}`


•Total revenue increased $203 million to $8.2 billion compared to 2022, driven by an increase of 4% in net interest income, including the
impacts of the HSBC transaction and Investors acquisition.
•The efficiency ratio of 67.0% compared to 61.0% in 2022.
◦On an Underlying basis, the efficiency ratio of 60.8% compared to 57.5% in 2022.
•ROTCE of 10.9% compared to 13.9% in 2022.
◦On an Underlying basis, ROTCE of 13.5% compared to 16.4%.
•Tangible book value per common share of $30.91 increased 11% from 2022.
For additional information regarding our financial performance, see “Results of Operations — 2023 compared with 2022” included in this
report.(1) (2) (3)
(1) (2) (4)
(1)
(2) 
(3) 
(4)
Citizens Financial Group, Inc. | 39RESULTS OF OPERATIONS — 2023 compared with 2022
Net Interest Income
Net interest income is our largest source of revenue and is the difference between the interest earned on i

{'input': 'what is the ROE for 2023?',
 'chat_history': [HumanMessage(content="ROE can be calculated using the formula net income divided by shareholder's equity"),
  AIMessage(content="Sure, I'll calculate ROE using this formula, ROE = net income of the company/shareholder's equity of the company")],
 'output': "I apologize, but I couldn't find the specific information about the ROE for 2023. The provided information mainly focuses on net interest income, efficiency ratio, ROTCE (Return on Tangible Common Equity), and tangible book value per common share. If you have any other financial data or specific information, I would be happy to assist you further in calculating the ROE for 2023."}

In [46]:
from langchain_core.prompts import ChatPromptTemplate

chat_template = ChatPromptTemplate.from_messages(
    [
        ("system", "These are some of the formulas for calcualting companies financial statement. ROE = Net income/Shareholder's equity. ROE ratio = (Net income/shareholder's equity)*100"),
        ("human", "Hello, how are you doing?"),
        ("ai", "I'm doing well, thanks!"),
        ("human", "{user_input}"),
    ]
)

messages = chat_template.format_messages(user_input="How to calcualte ROE?")

In [83]:
messages

[SystemMessage(content="These are some of the formulas for calcualting companies financial statement. ROE = Net income/Shareholder's equity. ROE ratio = (Net income/shareholder's equity)*100"),
 HumanMessage(content='Hello, how are you doing?'),
 AIMessage(content="I'm doing well, thanks!"),
 HumanMessage(content='How to calcualte ROE?')]

In [92]:
from langchain.agents import create_tool_calling_agent

agent = create_tool_calling_agent(chat_model, tools, messages
                                )

AttributeError: 'list' object has no attribute 'input_variables'

In [45]:
from langchain_core.messages import SystemMessage
from langchain_core.prompts import HumanMessagePromptTemplate

chat_template = ChatPromptTemplate.from_messages(
    [
        SystemMessage(
            content=(
                "You are a helpful assistant and use the following formulas to calcualte financial terms. "
                "ROE = Net income/Shareholder's equity."
                "ROE ratio = (Net income/shareholder's equity)*100"
            )
        ),
        HumanMessagePromptTemplate.from_template("{text}"),
    ]
)
messages = chat_template.format_messages(text="what is the formula for calculating ROE?")
print(messages)

NameError: name 'ChatPromptTemplate' is not defined

In [42]:
from langchain.chains import create_citation_fuzzy_match_chain

In [43]:
chain = create_citation_fuzzy_match_chain(chat_model)

In [47]:
result = chain.run(question=messages, context=docs)

In [49]:
print(result)

question='How to calculate ROE?' answer=[FactWithEvidence(fact="ROE = Net income/Shareholder's equity", substring_quote=["ROE = Net income/Shareholder's equity."]), FactWithEvidence(fact="ROE ratio = (Net income/shareholder's equity)*100", substring_quote=["ROE ratio = (Net income/shareholder's equity)*100"])]


In [ ]:
[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [50]:
%pip install llama-index-embeddings-openai
%pip install llama-index-llms-openai

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [1]:
!pip install llama-index

Defaulting to user installation because normal site-packages is not writeable


In [2]:
import os
from llama_index.llms.openai import OpenAI
from llama_index.core.query_engine import CitationQueryEngine
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core import (
    VectorStoreIndex,
    SimpleDirectoryReader,
    StorageContext,
    load_index_from_storage,
)

In [ ]:
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.llms.openai import OpenAI
from llama_index.core import Settings

Settings.llm = OpenAI(model="gpt-3.5-turbo")
Settings.embed_model = OpenAIEmbedding(model="text-embedding-3-small")

In [ ]:
# retriever
# model

In [19]:
from langchain_core.runnables import RunnableParallel
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

In [23]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [20]:
rag_chain_from_docs = (
    RunnablePassthrough.assign(context=(lambda x: format_docs(x["context"])))
    | prompt
    | model
    | StrOutputParser()
)

In [21]:
rag_chain_with_source = RunnableParallel(
    {"context": retriever, "question": RunnablePassthrough()}
).assign(answer=rag_chain_from_docs)

In [25]:
response = rag_chain_with_source.invoke("what is the net income of the company for 2023?")

In [35]:
print(response)

{'context': [Document(page_content='Year Ended December 31, 2023\n(dollars in millions)Consumer\nBankingCommercial\nBanking Non-Core Other Consolidated\nNet interest income $4,187 $2,292 ($129) ($109) $6,241 \nNoninterest income 1,067 784 — 132 1,983 \nTotal revenue 5,254 3,076 (129) 23 8,224 \nNoninterest expense 3,542 1,295 123 547 5,507 \nProfit (loss) before provision (benefit) for credit losses 1,712 1,781 (252) (524) 2,717 \nProvision (benefit) for credit losses 280 250 78 79 687 \nIncome (loss) before income tax expense (benefit) 1,432 1,531 (330) (603) 2,030 \nIncome tax expense (benefit) 373 378 (86) (243) 422 \nNet income (loss) $1,059 $1,153 ($244) ($360) $1,608 \nTotal average assets $72,693 $76,028 $13,745 $59,755 $222,221 \nYear Ended December 31, 2022\n(dollars in millions)Consumer\nBankingCommercial\nBanking Non-Core Other Consolidated\nNet interest income $3,649 $2,103 $378 ($118) $6,012 \nNoninterest income 1,063 845 — 101 2,009 \nTotal revenue 4,712 2,948 378 (17) 8,